#**ETL Pipeline(Extract, Transform and Load)**

In [3]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
import random
from faker import Faker
fake = Faker()

#large dataset with 1000 rows
data = {
    "id": list(range(1, 1001)),
    "name": [fake.first_name() for _ in range(1000)],
    "age": [random.randint(18, 65) for _ in range(1000)],
    "city": [fake.city() for _ in range(1000)],
    "salary": [random.randint(30000, 150000) for _ in range(1000)]
}


df = pd.DataFrame(data)
df.to_csv("large_data.csv", index=False)
print("Large dataset CSV file created successfully!")


Large dataset CSV file created successfully!


**Extracting the data**


In [6]:

df = pd.read_csv("large_data.csv")
print("Extracted Data:")
print(df.head())


Extracted Data:
   id   name  age              city  salary
0   1  Chase   25         Davidside   52357
1   2  Marie   18     West Maryfurt  129727
2   3  Aaron   41       New Brandon   34943
3   4  David   56  East Christopher   74770
4   5  David   20      Marshborough   67844


**Transform Data (Cleaning, Filtering, Aggregations)**

Removing duplicates

Filtering records where age > 30

Creating a new column "tax" as 10% of salary

Aggregating average salary by city

In [8]:
df = df.drop_duplicates()

#Keep only rows where age > 30
df_filtered = df[df["age"] > 30]

#Adding a new column 'tax' (10% of salary)
df_filtered["tax"] = df_filtered["salary"] * 0.10

#Find average salary per city
df_aggregated = df_filtered.groupby("city")["salary"].mean().reset_index()

df.to_csv("large_data.csv", index=False)
print("Large dataset CSV file created successfully!")
print("Filtered Data:")
print(df_filtered)
print("\nAggregated Data (Avg Salary by City):")
print(df_aggregated)


Large dataset CSV file created successfully!
Filtered Data:
       id      name  age              city  salary      tax
2       3     Aaron   41       New Brandon   34943   3494.3
3       4     David   56  East Christopher   74770   7477.0
5       6      John   57  West Victorville  141308  14130.8
7       8    Taylor   38    South Lydiaton   49102   4910.2
8       9      Luke   64         Jamesfort   74223   7422.3
..    ...       ...  ...               ...     ...      ...
994   995     Shawn   31         West Ryan   97049   9704.9
995   996    Robert   33        Watersfort   84535   8453.5
996   997  Gabriela   47         Jonesland   96966   9696.6
998   999      Tony   37       Ramirezbury  139900  13990.0
999  1000    Tricia   43      Patrickmouth  142387  14238.7

[755 rows x 6 columns]

Aggregated Data (Avg Salary by City):
              city    salary
0        Aaronberg  131231.0
1        Adammouth   41541.0
2         Adamport  113081.0
3    Alexandraside   30664.0
4       Aman

<ipython-input-8-fdd5c29deb61>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["tax"] = df_filtered["salary"] * 0.10


**Load Data into SQLite**

In [11]:
import sqlite3
df = pd.read_csv("large_data.csv")


# Connect to SQLite database (creates file if not exists)
conn = sqlite3.connect("etl_database.db")
cursor = conn.cursor()

# Create a table in SQLite
cursor.execute('''
    CREATE TABLE IF NOT EXISTS employees (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        city TEXT,
        salary REAL,
        tax REAL
    )
''')

#Insert data into the employees table
df.to_sql("employees", conn, if_exists="replace", index=False)

print("Table created successfully and data loaded into the SQLite database, Hurrahh!!!")




Table created successfully and data loaded into the SQLite database, Hurrahh!!!


**Checking the data loaded successfully in SQLite**

In [12]:
#Reading the data from the SQLite table
df_sql = pd.read_sql("SELECT * FROM employees", conn)
print("\nData from SQLite Database:")
print(df_sql)


Data from SQLite Database:
       id      name  age              city  salary
0       1     Chase   25         Davidside   52357
1       2     Marie   18     West Maryfurt  129727
2       3     Aaron   41       New Brandon   34943
3       4     David   56  East Christopher   74770
4       5     David   20      Marshborough   67844
..    ...       ...  ...               ...     ...
995   996    Robert   33        Watersfort   84535
996   997  Gabriela   47         Jonesland   96966
997   998    Sheila   30  Rodriguezchester   85172
998   999      Tony   37       Ramirezbury  139900
999  1000    Tricia   43      Patrickmouth  142387

[1000 rows x 5 columns]


In [13]:
conn.close()